# IMDB Recommender System

## Data Preparation and Initialization

### Import Packages

In [ ]:
import os
import zipfile
import pandas
import csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
import scipy
from implicit import bpr
from sklearn import linear_model
import numpy


### Dataset Preparation

In [ ]:
archive = zipfile.ZipFile('dataset.zip', 'r')
archive.extractall()

In [ ]:
# Get the list of files in the archive
moive_genre = []
movie_reviews = []
for filename in archive.namelist():
    if "1_movies_per_genre" in filename:
        moive_genre.append(filename)
    else:
        movie_reviews.append(filename)
moive_genre.pop(0)
movie_reviews.pop(0)

In [ ]:
# Read the data from the files
data_dict = {}
movie_dict = {}
for f in movie_reviews:
    with open(f, mode ='r') as file:
        csv_reader = csv.DictReader(file)
        data_dict[f[24:-9]] = [row for row in csv_reader]

genre_dict = {}
for f in moive_genre:
    with open(f, mode ='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            genre_dict[f[19:-4]] = row
            movie_dict[row['name']] = row

### Data Preprocessing

In [ ]:
# Create a dictionary to store the data for each movie and each user
moviesPerUser_raw = defaultdict(set)
usersPerMovie_raw = defaultdict(set)
ratingPerUser = defaultdict(list)
ratingPerMovie = defaultdict(list)
yearRating = defaultdict(list)

# Create a set of all user-movie pairs
userMoviePair = set()
users = set()
movies = set()
data = []

# Create a dictionary to store the betaU and betaI values for each user and each movie
# BetaU and BetaI are the average ratings given by each user and each movie
betaU_dict = {}
betaI_dict = {}
movie_ratings_raw = []
userIDs, movieIDs = {}, {}

for movie, comments in data_dict.items():
    movies.add(movie)
    if not movie in movieIDs: movieIDs[movie] = len(movieIDs)
    for comment in comments:
        user = comment["username"]
        data.append(comment)
        data[-1]["moviename"] = movie
        users.add(user)
        moviesPerUser_raw[user].add(movie)
        usersPerMovie_raw[movie].add(user)
        if not user in userIDs: userIDs[user] = len(userIDs)
        userMoviePair.add((user, movie))
        if comment["rating"] != "Null":
            rating = int(comment["rating"])
            yearRating[comment["date"][-4:]].append(rating)
            ratingPerUser[user].append(rating)
            ratingPerMovie[movie].append(rating)
            betaU_dict[user] = [betaU_dict.get(user, [0.0, 0])[0] + rating, betaU_dict.get(user, [0.0, 0])[1] + 1]
            betaI_dict[movie] = [betaI_dict.get(movie, [0.0, 0])[0] + rating, betaI_dict.get(movie, [0.0, 0])[1] + 1]
            movie_ratings_raw.append([user, movie, rating])

In [ ]:
# Filter out users who have rated less than 3 movies
thres = 3

movie_ratings = []
moviesPerUser = defaultdict(set)
usersPerMovie = defaultdict(set)

for u in moviesPerUser_raw:
    if len(moviesPerUser_raw[u]) > thres:
        moviesPerUser[u] = moviesPerUser_raw[u]

for item in movie_ratings_raw:
    if item[0] in moviesPerUser:
        movie_ratings.append(item)
        usersPerMovie[item[1]].add(item[0])
        
random.shuffle(movie_ratings)

In [ ]:
# Create a dictionary to store the data for each movie and each genre
moviesPerGenre = defaultdict(set)
genresPerMovie = defaultdict(set)
genres = set()

for genre, movies in genre_dict.items():
    genres.add(genre)
    for movieinfo in movies:
        movie = movieinfo['name']
        moviesPerGenre[genre].add(movie)
        genresPerMovie[movie].add(genre)

### Data Analysis

#### Number of Movies per Genre

In [ ]:
x = []
y = []
for key, value in moviesPerGenre.items():
    x.append(key)
    y.append(len(value))

fig = plt.figure(figsize = (20, 5))
plt.bar(x, y, color ='blue', width = 0.5)
plt.xlabel("Genre")
plt.ylabel("No. of Movies")
plt.title("Number of Movies per Genre")
plt.show()

#### Number of People for Each Rating

In [ ]:
y = [0 for i in range(10)]
x = [(i+1) for i in range(10)]

for comment in data_dict["The Dark Knight"]:
    if comment["rating"] != "Null":
        y[int(comment["rating"])-1] += 1

fig = plt.figure(figsize = (10, 5))
plt.bar(x, y, color ='blue', width = 0.5)
plt.xlabel("Rating")
plt.ylabel("Number of Ratings")
plt.title("Number of People for Each Rating")
plt.show()

#### Average of Movie Rating Each Year

In [ ]:
x = []
y = []

items = sorted(list(yearRating.items()))
for year, rating in items:
    x.append(year)
    y.append(sum(rating)/len(rating))

fig = plt.figure(figsize = (20, 5))
plt.bar(x, y, color ='blue', width = 0.5)
plt.xlabel("Years")
plt.ylabel("Average of Movie Rating")
plt.title("Average of Movie Rating Each Year")
plt.show()

#### Average of Genre Rating

In [ ]:
def addlabels(x, y, fontSize=8, height=0.1):
    y = [round(num, 2) for num in y]
    for i in range(len(x)):
        plt.text(i, y[i]+height, y[i], ha = 'center', fontsize=fontSize)

In [ ]:
genre_rating = {}
for gen in genres:
    avg_rating = 0.
    for movie in genre_dict[gen]:
        avg_rating += float(movie['rating'])
    genre_rating[gen] = avg_rating / len(genre_dict[gen])

x = range(len(genre_rating))
y = list(genre_rating.values())

fig, ax = plt.subplots(figsize=(20, 5))
plt.bar(x, y, tick_label=list(genre_rating.keys()), color ='#3081D0', width = 0.5)
addlabels(x, y)
plt.xlabel("Genre", fontsize=15)
plt.ylabel("Average", fontsize=15)
plt.title("Average of Genre Rating", fontsize=20)
plt.show()

#### Average Movie Length Each Year

In [ ]:
yearLength = defaultdict(list)
for gen in genre_dict.values():
    for movie in gen:
        if len(movie['run_length']) == 2:
            movie_length = datetime.datetime.strptime(movie['run_length'], '%Hh').time()
        else:
            movie_length = datetime.datetime.strptime(movie['run_length'], '%Hh %Mmin').time()
        yearLength[movie['year']].append(movie_length)

yearAvgLen = {}
for year, time in yearLength.items():
    minSum = 0
    for t in time:
        minSum += t.hour * 60 + t.minute
    yearAvgLen[int(year)] = minSum / len(time)

y = []
label = []
for key in sorted(yearAvgLen.keys()):
    if key < 1999:
        continue
    y.append(yearAvgLen[key])
    label.append(key)
x = range(len(y))

fig, ax = plt.subplots(figsize=(20, 5))
plt.bar(x, y, tick_label=label, color='#3081D0', width=0.5)
addlabels(x, y, 8, 2)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Average Movie Length", fontsize=15)
plt.title("Average Movie Length Each Year", fontsize=20)
plt.show()

## Rating Prediction

### Baseline Model
The prediction function is defined as  
$f(user, item) = \alpha + \beta_u + \beta_i$,  
where $\alpha$ is a constant, $\beta_u$ is how much does this user tend to rate things above the mean, and $\beta_i$ is how much does this item tend to receive  ratings than others.
  
The optimization problem:  
$\argmin_{\alpha, \beta} \sum_{u, i} (\alpha + \beta_u + \beta_i - R_{u,i})^2 + \lambda [\sum_u \beta_u^2 + \sum_i \beta_i^2]$,  
where $R_{u,i}$ is the ground truth rating.  
The first term and the second term of the equation is error and regularizer, respectively.

#### Construct Beta for each user and movie

In [ ]:
betaU = {}
betaI = {}

random.shuffle(movie_ratings)
ratings = [r[-1] for r in movie_ratings]
globalAverage = sum(ratings) * 1.0 / len(ratings)

for u in betaU_dict:
    betaU[u] = betaU_dict[u][0] / betaU_dict[u][1] - globalAverage

for m in betaI_dict:
    betaI[m] = betaI_dict[m][0] / betaI_dict[m][1] - globalAverage

alpha = globalAverage

#### Optimize the Parameters
By differentiating,  
$\alpha = \frac{\sum_{u,i\in train}R_{u,i} - (\beta_u + \beta_i)}{N_{train}}$  
$\beta_u = \frac{\sum_{i\in I_u} R_{u,i} - (\alpha + \beta_i)}{\lambda + |I_u|}$  
$\beta_i = \frac{\sum_{u\in U_i} R_{u,i} - (\alpha + \beta_u)}{\lambda + |U_i|}$

##### Model Training

In [ ]:
def iterate(alpha, betaU, betaI, lamb=1, iteration=5):
    for _ in range(iteration):
        betaU_new = {}
        betaI_new = {}
        alpha_new = 0.0
        ratingPerUser = {}
        ratingPerItem = {}
        thres = 1e-3

        for u, m, rating in movie_ratings[:int(len(movie_ratings)*0.8)]:
            alpha_new += (rating - betaU[u] - betaI[m])

        ###  update alpha  ###
        alpha_new /= len(movie_ratings[:int(len(movie_ratings)*0.8)])
        alpha = alpha_new
        
        for u, m, rating in movie_ratings[:int(len(movie_ratings)*0.8)]:
            ratingPerUser[u] = [ratingPerUser.get(u, [0.0, 0])[0] + (rating - alpha - betaI[m]), ratingPerUser.get(u, [0.0, 0])[1] + 1]
            
        ###  update betaU  ###
        for u in ratingPerUser:
            betaU_new[u] = ratingPerUser[u][0] / (lamb + ratingPerUser[u][1])
        
        betaU = betaU_new
        
        for u, m, rating in movie_ratings[:int(len(movie_ratings)*0.8)]:
            ratingPerMovie[m] = [ratingPerMovie.get(m, [0.0, 0])[0] + (rating - alpha - betaU[u]), ratingPerMovie.get(m, [0.0, 0])[1] + 1]
        ###  update betaI  ###
        for m in ratingPerMovie:
            betaI_new[m] = ratingPerMovie[m][0] / (lamb + ratingPerMovie[m][1])
            

        alpha, betaU, betaI = alpha_new, betaU_new, betaI_new
    
    return alpha_new, betaU_new, betaI_new

#### Helper Function

In [ ]:
def clip(n):
    n = round(n)
    n = max(n, 1)
    n = min(n, 10)
    return n

In [ ]:
def mse(alpha, betaU, betaI):
    mse = 0.0
    for u, m, rating in movie_ratings[int(len(movie_ratings)*0.8):]:
        if u in betaU and m in betaI:
            mse += (rating - clip(alpha + betaU[u] + betaI[m]))**2
        elif m in betaI:
            mse += (rating - clip(alpha - betaI[m]))**2
        else:
            mse += (rating - clip(alpha))**2
    mse /= len(movie_ratings[int(len(movie_ratings)*0.8):])
    return mse

In [ ]:
for i in range(5):
    alpha, betaU, betaI = iterate(alpha, betaU, betaI, 2, 5)
    validMSE = mse(alpha, betaU, betaI)
    print("mse:", validMSE)

In [ ]:
print(len(movie_ratings[int(len(movie_ratings)*0.8):]))

### Ridge Model

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

X = [[alpha, betaU[u], betaI[m]] for u, m, rating in movie_ratings]
y = ratings

ridge=Ridge()
parameters={'alpha':[0,1e-3,1e-2,1,5]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(X,y)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

In [ ]:
pred = ridge_regressor.predict(X)
mse = 0.0
for i in range(len(pred)):
    mse += (y[i] - round(pred[i]))**2
mse /= len(movie_ratings)
print(mse)

### SVD

In [ ]:
import pandas as pd
from surprise import BaselineOnly, SVD, SVDpp, Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

kf = KFold(n_splits=3)

min_rating, max_rating = 1, 10

ratings_dict = {"userID": [], "itemID": [], "ratings": []}
for l in movie_ratings:
    ratings_dict["userID"].append(l[0])
    ratings_dict["itemID"].append(l[1])
    ratings_dict["ratings"].append(l[2])

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(min_rating, max_rating))
data_df = Dataset.load_from_df(df[["userID", "itemID", "ratings"]], reader)
trainset, testset = train_test_split(data_df, test_size=0.2)
# model = BaselineOnly()
model = SVD()

#for trainset, testset in kf.split(data_df):

# train and test algorithm.
model.fit(trainset)
predictions = model.test(testset)

# Compute and print Root Mean Squared Error
accuracy.mse(predictions, verbose=True)

In [ ]:
trainset, testset = train_test_split(data_df, test_size=0.2)
pred = model.test(testset)

mse = 0.0
for i in range(len(pred)):
    mse += (pred[i][2] - clip(pred[i][3]))**2
mse /= len(pred)
print(mse)

accuracy.mse(pred, verbose=True)

## Would Watch Prediction

#### Jaccard Similarity

In [ ]:
def Jaccard(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

### Baseline Model
Predict the user would watch the movie by logic

In [ ]:
def isPlayed(u, i, func, thres):
    similarities = []
    users = usersPerMovie[i]
    candidateMovies = set()
    for u in users:
        candidateMovies = candidateMovies.union(moviesPerUser[u])
    for i2 in candidateMovies:
        if i2 == i: continue
        sim = func(users, usersPerMovie[i2])
        similarities.append(sim)
    if not similarities:
        return False
    similarities.sort()

    # return similarities[-1] >= thres

    return statistics.mean(similarities[-3:]) > thres

In [ ]:
def isPlayed(u, i, func, thres):
    similarities = []
    users = usersPerMovie[i]
    for j in moviesPerUser[u]:
        if i == j: continue
        sim = func(set(users), set(usersPerMovie[j]))
        similarities.append(sim)
    if not similarities:
        return False
    similarities.sort()

    # return similarities[-1] >= thres

    return statistics.mean(similarities[-3:]) > thres

In [ ]:
samples = []
flaseSampleNum = 1
falseUserMoviePair = set()
for user, movie in userMoviePair:
    samples.append([(user, movie), True])

    for _ in range(flaseSampleNum):
        n = 0
        while True:
            false_movie = random.choices([*movies],k=1)[0]['name']
            false_sample = (user, false_movie)
            if (false_sample not in userMoviePair) and (false_sample not in falseUserMoviePair) or n > 50:
                break
            n += 1
        falseUserMoviePair.add(false_sample)
        samples.append([false_sample, False])

In [ ]:
random.seed(666)
random.shuffle(samples)
trian_sample = samples[:int(0.9*len(samples))]
valid_sample = samples[int(0.9*len(samples)):]

In [ ]:
test = [len(usersPerMovie[s[0][1]]) for s in trian_sample]
print(statistics.median(test))

In [ ]:
correct = 0
thres = 0.1
popular_thres = 1000

for s in trian_sample:
    played = isPlayed(s[0][0], s[0][1], Jaccard, thres)
    popular = len(usersPerMovie[s[0][1]]) > popular_thres
    if (s[1] and played) or (s[1] and popular) or (not s[1] and not played and not popular):
        correct += 1
print(correct / len(samples))

In [ ]:
print(correct / len(samples))

#### Bayesian Personalized Ranking

In [ ]:
Xui = scipy.sparse.lil_matrix((len(userIDs), len(movieIDs)))
for pair, label in samples:
    if not label:
        continue
    Xui[userIDs[pair[0]], movieIDs[pair[1]]] = 1

Xui_csr = scipy.sparse.csr_matrix(Xui)

In [ ]:
model = bpr.BayesianPersonalizedRanking()
model.fit(Xui_csr)

In [ ]:
correct = 0
num_recommended = 1000

for s in samples:
    recommended = model.recommend(userIDs[u], Xui_csr[userIDs[u]], N=num_recommended)[0]
    rcmSet = {sam for sam in recommended}
    if (s[1] and s[0] in rcmSet) or (not s[1] and s[0] not in rcmSet):
        correct += 1
print(correct / len(samples))

#### Logistic

In [ ]:
def BER(pred, ground_truth):
    TP_ = numpy.logical_and(pred, ground_truth)
    FP_ = numpy.logical_and(pred, numpy.logical_not(ground_truth))
    TN_ = numpy.logical_and(numpy.logical_not(pred), numpy.logical_not(ground_truth))
    FN_ = numpy.logical_and(numpy.logical_not(pred), ground_truth)

    TP = sum(TP_)
    FP = sum(FP_)
    TN = sum(TN_)
    FN = sum(FN_)

    return 1 - 0.5*(TP / (TP + FN) + TN / (TN + FP))

In [ ]:
def maxSim(u, i, func):
    similarities = []
    users = usersPerMovie[i]
    for j in moviesPerUser[u]:
        if i == j: continue
        sim = func(set(users), set(usersPerMovie[j]))
        similarities.append(sim)
    if not similarities:
        return False
    similarities.sort()

    return similarities[-1]

In [ ]:
max_viewer = -1
for _, val in usersPerMovie.items():
    max_viewer = max(max_viewer, len(val))

nor_popular = {}
for movie, val in usersPerMovie.items():
    nor_popular[movie] = len(val) / max_viewer

In [ ]:
xtrain = [[1, nor_popular[s[0][1]], maxSim(s[0][0], s[0][1], Jaccard)] for s in trian_sample]
ytrain = [s[1] for s in trian_sample]

xvalid = [[1, nor_popular[s[0][1]], maxSim(s[0][0], s[0][1], Jaccard)] for s in valid_sample]
yvalid = [s[1] for s in valid_sample]

In [ ]:
best_acc = 0.
best_BER = 50
best_c = None
max_it = 10000
solver = 'lbfgs'

for reg in [0.001, 0.1, 1, 5, 10, 100, 200]:
    mod = linear_model.LogisticRegression(C=reg, class_weight = 'balanced', max_iter=max_it, solver=solver)
    mod.fit(xtrain, ytrain)

    predValid = mod.predict(xvalid)
    acc = sum(predValid == yvalid) / len(yvalid)
    ber = BER(predValid, yvalid)

    if best_BER > ber:
        best_BER = ber
        best_acc = acc
        best_c = reg
        print(reg, acc, ber)

print(best_c, best_acc, ber)

#### Nueral Network

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
num_genre = len(genre_dict)

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.layer1_1 = nn.Linear((num_genre + 5), 128)
        self.layer1_2 = nn.Linear((num_genre + 5), 128)
        self.layer2_1 = nn.Linear(128, 256)
        self.layer2_2 = nn.Linear(128, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64) 
        self.layer5 = nn.Linear(64, 16) 
        self.layer6 = nn.Linear(64, 2) 

        self.act_fn = nn.ReLU()

    def forward(self, x1, x2):
        x1 = self.layer1_1(x1)
        x1 = self.act_fn(x1)

        x2 = self.layer1_2(x2)
        x2 = self.act_fn(x2)

        x1 = self.layer2_1(x1)
        x1 = self.act_fn(x1)

        x2 = self.layer2_2(x2)
        x2 = self.act_fn(x2)

        x = self.layer3(x1+x2)
        x = self.act_fn(x)
        
        x = self.layer4(x)
        x = self.act_fn(x)

        x = self.layer5(x)
        x = self.act_fn(x)

        out = self.layer6(x)
        
        return out

In [ ]:
genreID = {}
for genre in genre_dict.keys():
    if genre not in genreID:
        genreID[genre] = len(genreID)

In [ ]:
movie_dict['10 Cloverfield Lane']

In [ ]:
def features_user(user):
    fea = [0] * (num_genre + 5)
    fea[-1] = 1
    for movie in moviesPerUser[user]:
        for genre in movie_dict[movie]['genres'].split(';'):
            if genre in genreID:
                fea[genreID[genre]] = 1
        fea[num_genre] += int(movie_dict[movie]['year'])
        fea[num_genre+1] += float(movie_dict[movie]['rating'])
        fea[num_genre+2] += float(movie_dict[movie]['num_raters'])
        fea[num_genre+3] += float(movie_dict[movie]['num_reviews'])
    for i in range(len(fea)):
        fea[i] /= len(moviesPerUser[user])
    
    return fea

In [ ]:
def features_movie(movie):
    fea = [0] * (num_genre + 5)
    fea[-1] = 1
    for genre in movie_dict[movie]['genres'].split(';'):
        if genre in genreID:
            fea[genreID[genre]] = 1
    fea[num_genre] += int(movie_dict[movie]['year'])
    fea[num_genre+1] += float(movie_dict[movie]['rating'])
    fea[num_genre+2] += float(movie_dict[movie]['num_raters'])
    fea[num_genre+3] += float(movie_dict[movie]['num_reviews'])
    
    return fea

In [ ]:
class movieDataset(Dataset):
    def __init__(self, x1, x2, y):
        self.data = data
        self.x1 = torch.FloatTensor(x1.copy())
        self.x2 = torch.FloatTensor(x2.copy())
        self.y = torch.FloatTensor(y.copy())
        
    def __getitem__(self, index):
        return self.x1[index], self.x2[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [ ]:
x1train_nn = [features_user(s[0][0]) for s in trian_sample]
x2train_nn = [features_movie(s[0][1]) for s in trian_sample]
ytrain_nn = [s[1] for s in trian_sample]

x1valid_nn = [features_user(s[0][0]) for s in trian_sample]
x2valid_nn = [features_movie(s[0][1]) for s in trian_sample]
yvalid_nn = [s[1] for s in valid_sample]

In [ ]:
# Train
def train(model, config, device):
    n_epochs = config['n_epochs']  # Maximum number of epochs

    dataset = movieDataset(x1train_nn, x2train_nn, ytrain_nn)

    # Datasets
    dataseloader = DataLoader(dataset, config['batch_size'], shuffle=True,
                         drop_last=False, pin_memory=True)

    # Optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(model.parameters(), **config['optim_hparas'])
    
    # Learning rate scheduler
    lambda1 = lambda epoch: config['lambda'] ** epoch
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
    
    model.to(device)

    best_loss = 10000.
    best_acc = 0.
    epoch = 0
    criterion = nn.CrossEntropyLoss()

    model.train()
    while epoch < n_epochs:
        print(f'Epoch: {epoch}')
        running_loss = 0.
        running_acc = 0.

        for x1, x2, y in dataseloader:
            optimizer.zero_grad()
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)

            with torch.set_grad_enabled(True):
                pred = model(x1, x2)
                loss = criterion(pred, y)

                _, preds = torch.max(pred, 1)

                loss.backward()
                optimizer.step()

            running_loss += loss
            running_acc += torch.sum(preds == y).item()
        print(f'Loss: {running_loss} Acc: {running_acc/len(dataloader.dataset)}')
        
        # schedule learning rate
        scheduler.step()

        if running_loss < best_loss:
            best_loss = running_loss
            torch.save(model.state_dict(), config['save_path'])
            print('Model saved!')
        epoch += 1

In [ ]:
config = {
    'n_epochs': 5000,
    'batch_size': 4096,
    'optimizer': 'Adam',
    'optim_hparas': {
        'lr': 0.00001,
        'weight_decay': 0.001, 
    },
    'lambda': 0.996,
    'early_stop': 200,
    'save_path': 'models/model.pth'
}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
model = NN()
train(model, config, device)